In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

filename = '../input/ice-cream-dataset/combined/products.csv'
usecols = ['brand', 'key', 'name', 'subhead', 'description', 'rating', 'rating_count', 'ingredients']
df = pd.read_csv(filepath_or_buffer=filename, usecols=usecols, )
df.head()

,brand,key,name,subhead,description,rating,rating_count,ingredients
0,bj,0_bj,Salted Caramel Core,Sweet Cream Ice Cream with Blonde Brownies & a...,Find your way to the ultimate ice cream experi...,3.7,208,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
1,bj,1_bj,Netflix & Chilll'd™,Peanut Butter Ice Cream with Sweet & Salty Pre...,There’s something for everyone to watch on Net...,4.0,127,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
2,bj,2_bj,Chip Happens,A Cold Mess of Chocolate Ice Cream with Fudge ...,Sometimes “chip” happens and everything’s a me...,4.7,130,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,..."
3,bj,3_bj,Cannoli,Mascarpone Ice Cream with Fudge-Covered Pastry...,As a Limited Batch that captured the rapture o...,3.6,70,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),..."
4,bj,4_bj,Gimme S’more!™,Toasted Marshmallow Ice Cream with Chocolate C...,It’s a gimme: there’s always room for s’more. ...,4.5,281,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ..."


In [2]:
from re import sub
t0_columns = ['key', 'brand', 'name', 'ingredients']
t0_df = df[t0_columns].copy(deep=True)
t0_df['long name'] = t0_df.apply(func=lambda x: ' '.join([x['brand'], x['name']]), axis=1)
t0_df['short'] = t0_df['ingredients'].apply(func=lambda x: sub(r'\([^)]*\)', '', x).lower())
t0_df['list'] = t0_df['short'].str.split(',')
t0_df['list'] = t0_df['list'].apply(func=lambda x: [item.replace('and/or', ' ').replace(':', ' ').strip() for item in x])
t0_df.head()


,key,brand,name,ingredients,long name,short,list
0,0_bj,bj,Salted Caramel Core,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",bj Salted Caramel Core,"cream, skim milk, liquid sugar , water, brown ...","[cream, skim milk, liquid sugar, water, brown ..."
1,1_bj,bj,Netflix & Chilll'd™,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",bj Netflix & Chilll'd™,"cream, skim milk, liquid sugar , water, sugar,...","[cream, skim milk, liquid sugar, water, sugar,..."
2,2_bj,bj,Chip Happens,"CREAM, LIQUID SUGAR (SUGAR, WATER), SKIM MILK,...",bj Chip Happens,"cream, liquid sugar , skim milk, water, sugar,...","[cream, liquid sugar, skim milk, water, sugar,..."
3,3_bj,bj,Cannoli,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",bj Cannoli,"cream, skim milk, liquid sugar , water, corn ...","[cream, skim milk, liquid sugar, water, corn s..."
4,4_bj,bj,Gimme S’more!™,"CREAM, SKIM MILK, WATER, LIQUID SUGAR (SUGAR, ...",bj Gimme S’more!™,"cream, skim milk, water, liquid sugar , sugar,...","[cream, skim milk, water, liquid sugar, sugar,..."


In [3]:
t0_df  = pd.concat([t0_df, pd.get_dummies(data=t0_df['list'].explode())], axis=1)
t0_df.head()

,key,brand,name,ingredients,long name,short,list,acesulfame potassium,almond extract,almond milk,...,whole milk powder,whole wheat graham flour,whole wheat/graham flour,wine,xanthan gum,yeast extract,yellow 5,yellow 5 lake,yellow 6,yellow 6 lake
0,0_bj,bj,Salted Caramel Core,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",bj Salted Caramel Core,"cream, skim milk, liquid sugar , water, brown ...","[cream, skim milk, liquid sugar, water, brown ...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0_bj,bj,Salted Caramel Core,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",bj Salted Caramel Core,"cream, skim milk, liquid sugar , water, brown ...","[cream, skim milk, liquid sugar, water, brown ...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0_bj,bj,Salted Caramel Core,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",bj Salted Caramel Core,"cream, skim milk, liquid sugar , water, brown ...","[cream, skim milk, liquid sugar, water, brown ...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0_bj,bj,Salted Caramel Core,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",bj Salted Caramel Core,"cream, skim milk, liquid sugar , water, brown ...","[cream, skim milk, liquid sugar, water, brown ...",0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0_bj,bj,Salted Caramel Core,"CREAM, SKIM MILK, LIQUID SUGAR (SUGAR, WATER),...",bj Salted Caramel Core,"cream, skim milk, liquid sugar , water, brown ...","[cream, skim milk, liquid sugar, water, brown ...",0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
t0_df['brand'].unique()

array(['bj', 'hd', 'talenti', 'breyers'], dtype=object)

In [5]:
drop_columns = ['key', 'brand', 'name', 'ingredients', 'long name', 'short', 'list']
# t0_df.drop(columns=drop_columns).describe()
t1 = t0_df.drop(columns=drop_columns).sum().to_dict()
t1

{'acesulfame potassium': 12,
 'almond extract': 1,
 'almond milk': 2,
 'almonds': 16,
 'almonds roasted in vegetable oil': 2,
 'anhydrous milkfat': 2,
 'annatto': 24,
 'apple juice': 1,
 'artificial color': 1,
 'artificial flavor': 5,
 'artificial flavoring': 1,
 'artificial flavors': 1,
 'ascorbic acid': 1,
 'baking powder': 2,
 'baking soda': 75,
 'baking soda   calcium phosphate': 2,
 'baking soda. contains milk': 1,
 'balsamic vinegar': 1,
 'banana puree': 3,
 'bananas': 1,
 'barley malt': 5,
 'beet juice': 3,
 'belgian chocolate': 1,
 'black carrot concentrate': 1,
 'black cherries': 1,
 'black raspberries': 1,
 'black raspberry puree': 2,
 'blackberry juice concentrate': 1,
 'bleached wheat flour': 3,
 'blue 1': 1,
 'blue 1 lake': 2,
 'blue 2': 1,
 'blue 2 lake': 1,
 'blueberries': 3,
 'blueberry puree concentrate': 1,
 'bourbon': 3,
 'brown sugar': 28,
 'brown sugar†': 1,
 'butter': 72,
 'butter oil': 9,
 'butteroil': 12,
 'calcium carbonate': 2,
 'calcium phosphate': 2,
 'cane 